# Monte Carlo application: estimating probability of an event

Estimating probability of an event $A$ via Monte Carlo method is pretty simple. We need to select a number of trials $N$, reproduce $N$ samples from the sample space, and count $\#(N_A)$ the number of the event $A$ occurrence in the sequence of $N$ trials. Finally, the probability estimate looks like this:

$$
\widehat{P(A)} = \frac{\#(N_A)}{N}\,.
$$

It is also convenient to use indicator function of the event, which is

$$
I_A(\omega) = \left\{
\begin{array}{ll}
1,& \mbox{ if event A occurs at the experiment }\omega,\\
0,& \mbox{ if event A does not occur at the experiment }\omega
\end{array}
\right.,
$$

now in a sequence of experiments $\omega_1,\dots,\omega_N$ we have

$$
\widehat{P(A)} = \frac1{N}\sum_{i=1}^N I_A(\omega_i),
$$

or simply the mean indicator function value.

## Calculating $\pi$

### Problem statement

Calculating $\pi$ is a very simple example of Monte Carlo method application. As we know, if a circle raduis is $r$, its area equals to $\pi r^2$. Consider a square with unit side and a quarter of unit circle inside as shown in the picture below. Areas ratio of the circle segment and the square equals to $\pi / 4$, so define an event $A$ as falling inside the circle segment a point drawn at random in the square. The event probability $P(A)=\pi/4$, so we may estimate $\pi$ by

$$
\widehat{\pi} = 4 * \widehat{P(A)}
$$

with $\widehat{P(A)}$ obtained by the Monte Carlo method described above.

<center>
<img src="img/prob_event_12.png" width="700" height="320">
</center>

### Calculating

In [1]:
# import the numeric library needed for calculations
import numpy as np

# select number of trials
N = 1000000

# create a random sample from the uniform distribution in the unit square
x = np.random.uniform(size=(1000000, 2))

# calculate indicator function of the event A values
ind_func_values = (np.linalg.norm(x, axis=1) <= 1)

# calculate indicator function mean
prob_estim = ind_func_values.mean()

# calculate pi extimate
pi_estim = 4 * prob_estim

# print the result
print('pi value estimate = {0:.4f}'.format(pi_estim))

pi value estimate = 3.1392


## Probability for origin to lie inside a random tetrahedron

### Problem statement

In the [video](https://www.youtube.com/watch?v=OkmNXy7er84) the following problem has been considered: suppose a tetrahedron is selected ar random so that all its vertices lie in a sphere centered at the origin. What is the probability that the origin lies inside this tetrahedron?

The video explains analytical solution for the problem, it equals 1/8. Here we present a Monte Carlo solution, which approximates the analytical one.

### Uniform distribution on a unit sphere

Let $x, y$ be a horizontal plane, and $z$ directs upwards. Next for a vector $(x, y, z)$ on the unit sphere we denote $\alpha$ the angle between its projection to $x, y$ plane and the $x$ axis, and $\beta$ - the angle between the vector and the $z$ axis, see figure below. Note that $\alpha\in[0, 2\pi)$ and $\beta\in[-\pi/2,\pi/2]$.

For a random point to be distributed uniformly on the unit sphere, we should have uniform distibution of $\alpha$, and the distribution of $\beta$ with density, proportional to the distance of projection $(x,y)$ from the origin, which is equal to $\cos \beta$. Thus the density is

$$
f_\beta(t) = \frac{\cos t}2,\;t\in[-\pi/2, \pi/2],
$$

and the CDF equals to

$$
F_\beta(t) = \frac{1 + \sin t}2,\;t\in[-\pi/2, \pi/2].
$$

The quantile function appears to be

$$
F_\beta^{-1}(u) = {\rm arc}\sin(2u - 1),\;u\in[0, 1].
$$

Now we may calculate the cartesian coordinates of the random points

\begin{align*}
z &= \sin\beta,\\
r &= \cos\beta,\\
x &= r\cos\alpha,\\
y &= r\sin\alpha.
\end{align*}

The $\alpha$ and $\beta$ angles are shown in the fugires below.

<center>
<img src="img/prob_event_34.png" width="700" height="334">
</center>

### Event $A$ definition

Given 4 independent points $T_1 = (x_1, y_1, z_1), T_2 = (x_2, y_2, z_2), T_3 = (x_3, y_3, z_3), T_4 = (x_4, y_4, z_4)$ from the uniform distribution on a unit sphere, we need to find **baricentric** coordinates of the origin $(0, 0, 0)$ with respect to the points. The baricentric coordinates $\mu_1, \mu_2, \mu_3, \mu_4$ satisfy the following vector eqiation $\mu_1T_1 + \mu_2T_2 + \mu_3T_3 + \mu_4T_4 = 0$, and additionally the condition $\mu_1 + \mu_2 + \mu_3 + \mu_4 = 1$, thus leading to 4 scalar equations with respect to 4 unknowns:

$$
\left(
\begin{array}{cccc}
x_1&x_2&x_3&x_4\\
y_1&y_2&y_3&y_4\\
z_1&z_2&z_3&z_4\\
1&1&1&1
\end{array}
\right)
\left(
\begin{array}{c}
\mu_1\\
\mu_2\\
\mu_3\\
\mu_4
\end{array}
\right) =
\left(
\begin{array}{c}
0\\
0\\
0\\
1
\end{array}
\right)\,,
$$


Event $A$ occurs, if all baricentric coordinates are non-negative:
$$
A = \{\mu_1, \mu_2, \mu_3, \mu_4 \geq0\}.
$$

In this case the origin lies inside a tetrahedron.

### Monte Carlo solution

To estimate the probability, we simulate $N=$ one million samples of 4 independent uniform points on the unit sphere, solve the linear system for each sample, check the signs of all baricentric coordinates, and calculate the target ratio.

### Implementation

In [2]:
# quantile function
appf = lambda z: np.arcsin(2 * z - 1)

# number of Monte Carlon trials required
n_trials = 1000000

# dimension of the problem
dim = 3

In [3]:
# uniform values for "alpha" angle
alpha = 2 * np.pi * np.random.uniform(size=(n_trials, 1, dim + 1))

# non-uniform values for beta angle
u = np.random.uniform(size=(n_trials, 1, dim + 1))
beta = appf(u)
z = np.sin(beta)
r = np.cos(beta)
x = r * np.cos(alpha)
y = r * np.sin(alpha)

In [4]:
# left hand side matrices
a = np.concatenate((x, y, z), axis=1)
aaa = np.ones((n_trials, 1, dim + 1))
a = np.concatenate((a, aaa), axis=1)

# right hand side vectors
b = np.zeros((n_trials, dim))
bbb = np.ones((n_trials, 1))
b = np.concatenate((b, bbb), axis=1)

# solutions of the linear systems (baricentric coordinates for all n_trials samples)
x = np.linalg.solve(a, b)
x = x.reshape(n_trials, dim + 1)

# are all components non-negative (which means the origin is located inside the corresponding tetrahedron)
ind = (x >= 0).all(axis=1)

# probability (portion of tetrahedrons containing the origin inside)
# 'ind' stands for the indicator function of A,
# it contains 1 for event A occurrence, and 0 for the opposite, so mean calculates exactly the ratio we need
print('probability = {0:.4f}'.format(ind.mean()))

probability = 0.1245


We see that the estimated probability is pretty close to the theoretical value 1/8.

## Vectorization vs loops

Note also that this use of the Monte Carlo method includes solution of 1 million *different* linear systems of equations simultaneously. Solving it one by one would take much longer, as shown below.

In [5]:
print(a.shape, b.shape)

(1000000, 4, 4) (1000000, 4)


### Simultaneous solution

In [6]:
%timeit x = np.linalg.solve(a, b)

209 ms ± 4.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Solving in loop

In [7]:
def solve_in_loop(loop_size=1000):
    """ """
    xx = np.zeros((loop_size, 4))
    for i in range(loop_size):
        xx[i, :] = np.linalg.solve(a[i, :, :], b[i, :])
    return xx

In [8]:
%timeit xx = solve_in_loop(loop_size=1000000)

7.88 s ± 241 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Conclusion

Thus using loops increases time consumption by about 40 times.